### --- Step 1: Setup and Imports ---

In [1]:
import pandas as pd
import requests
import time
import datetime
import hopsworks
import exclude.key

# Hopsworks
HOPSWORKS_API_KEY = exclude.key.HOPSWORKS_API_KEY
FEATURE_GROUP_NAME = "bars_near_london_bridge"
FEATURE_GROUP_VERSION = 3

# Besttime API
BESTTIME_API_KEY_PRIVATE = exclude.key.BESTTIME_API_KEY_PRIVATE
BESTTIME_API_KEY_PUBLIC = exclude.key.BESTTIME_API_KEY_PUBLIC
BAR_LOCATION = 'London Bridge, London'
NUMBER_OF_BARS = 50

# Connect to Hopsworks
project = hopsworks.login(api_key_value=HOPSWORKS_API_KEY)
fs = project.get_feature_store()

Connected. Call `.close()` to terminate connection gracefully.

Logged in to project, explore it here https://c.app.hopsworks.ai:443/p/25749
Connected. Call `.close()` to terminate connection gracefully.


### --- Step 2: Create or Get Feature Group ---

In [2]:
def get_or_create_feature_group():
    try:
        fg = fs.get_feature_group(name=FEATURE_GROUP_NAME, version=FEATURE_GROUP_VERSION)
    except:
        fg = fs.create_feature_group(
            name=FEATURE_GROUP_NAME,
            version=FEATURE_GROUP_VERSION,
            description="Foot traffic data for bars near London Bridge",
            primary_key=['venue_name', 'day', 'hour'],
            event_time='last_updated',
            online_enabled=True
        )
    return fg


### --- Step 3: Perform Venue Search ---

In [3]:
def perform_venue_search(api_key=BESTTIME_API_KEY_PRIVATE):
    print("Performing venue search")
    endpoint = 'https://besttime.app/api/v1/venues/search'
    params = {
        'api_key_private': api_key,
        'q': f'bars near {BAR_LOCATION}',
        'num': NUMBER_OF_BARS,
        'format': 'all'  # Retrieve full forecast data
    }
    response = requests.post(endpoint, params=params)
    search_data = response.json()
    
    job_progress_url = search_data['_links']['venue_search_progress']
    print(f"Venue search initiated. Progress URL: {job_progress_url}")
    return job_progress_url


### --- Step 4: Retrieve Venue Search Results ---

In [4]:
def retrieve_venue_search_results(job_progress_url):
    print("Retrieving venue search results...")
    while True:
        response = requests.get(job_progress_url)
        progress_data = response.json()
        if progress_data['job_finished']:
            break
        time.sleep(5)
    venues = progress_data['venues']
    print(f"Found {len(venues)} venues.")
    return venues


### --- Step 5: Extract Historical Data ---

In [5]:
def extract_historical_data(venues):
    print("Extracting historical data...")
    historical_data = []
    for venue in venues:
        if venue['forecast']:  # Check if venue has forecast data
            forecast_data = venue['venue_foot_traffic_forecast']
            for day_data in forecast_data['analysis']:
                for hour_data in day_data['hour_analysis']:
                    historical_data.append([
                        venue['venue_name'],
                        venue['venue_address'],
                        day_data['day_info']['day_text'],
                        hour_data['hour'],
                        hour_data['intensity_txt']
                    ])
    return historical_data


### --- Step 6: Backfill Historical Data ---

In [10]:
def add_popularity_columns(df):
    # Encode 'Busyness' into numerical values
    busyness_mapping = {'Closed': 0, 'Low': 1, 'Below average': 2, 'Average': 3, 'Above average': 4, 'High': 5}
    df['busyness_numeric'] = df['busyness'].map(busyness_mapping)

    # Calculate popularity score as a weighted average of busyness levels
    df_popularity = df.groupby(['venue_name', 'venue_address']).agg({
        'busyness_numeric': ['mean', 'std']
    }).reset_index()

    df_popularity.columns = ['venue_name', 'venue_address', 'busyness_mean', 'busyness_std']

    # Normalize the popularity score
    df_popularity['popularity'] = df_popularity['busyness_mean'] / df_popularity['busyness_mean'].max()

    # Merge the popularity back into the historical data
    return df.merge(df_popularity[['venue_name', 'venue_address', 'popularity']], on=['venue_name', 'venue_address'])


def backfill_historical_data(api_key=BESTTIME_API_KEY_PRIVATE):
    print("Starting backfill")
    
    # Perform venue search and retrieve results
    job_progress_url = perform_venue_search(api_key)
    venues = retrieve_venue_search_results(job_progress_url)

    # Extract historical data from the venue search results
    historical_data = extract_historical_data(venues)

    # Convert to DataFrame
    df_historical = pd.DataFrame(historical_data, columns=['Venue Name', 'Venue Address', 'Day', 'Hour', 'Busyness'])

    # Rename columns to match feature store requirements
    df_historical.columns = ['venue_name', 'venue_address', 'day', 'hour', 'busyness']

    df_historical = add_popularity_columns(df_historical)

    # Add last_updated column
    df_historical['last_updated'] = datetime.datetime.now()

    # Get or create the feature group
    fg = get_or_create_feature_group()

    # Insert historical data into the feature group
    fg.insert(df_historical, write_options={"wait": True})
    print("Historical data backfilled successfully, with added label for busyness.")


### --- Step 7: Query Real-Time Data ---

In [11]:
def query_realtime_data(venue_id, api_key=BESTTIME_API_KEY_PRIVATE):
    endpoint = 'https://besttime.app/api/v1/venues/analysis'
    params = {
        'api_key_private': api_key,
        'venue_id': venue_id,
        'format': 'all'
    }
    response = requests.post(endpoint, params=params)
    return response.json()


### --- Step 8: Update Real-Time Data ---

In [14]:
def update_realtime_data(api_key=BESTTIME_API_KEY_PRIVATE):
    # Perform venue search and retrieve results
    job_progress_url = perform_venue_search(api_key)
    venues = retrieve_venue_search_results(job_progress_url)

    # Extract real-time data from the venue search results
    historical_data = extract_historical_data(venues)

    # Convert to DataFrame
    df_historical = pd.DataFrame(historical_data, columns=['Venue Name', 'Venue Address', 'Day', 'Hour', 'Busyness'])

    # Rename columns to match feature store requirements
    df_historical.columns = ['venue_name', 'venue_address', 'day', 'hour', 'busyness']

    df_historical = add_popularity_columns(df_historical)

    # Add last_updated column
    df_historical['last_updated'] = datetime.datetime.now()

    # Insert real-time data into the feature group
    fg = get_or_create_feature_group()
    fg.insert(df_historical, write_options={"wait": True})
    print("Real-time data updated successfully.")


### --- Step 9: Main Function ---

In [13]:
backfill_historical_data()

Starting backfill
Performing venue search
Venue search initiated. Progress URL: https://besttime.app/api/v1/venues/progress?job_id=14312edb-e4ad-4c83-890f-51496fdb57d9&collection_id=col_31fc130591944834a1c2f4bb9ae97fb9&format=all
Retrieving venue search results...
Found 34 venues.
Extracting historical data...


Uploading Dataframe: 0.00% |          | Rows 0/3696 | Elapsed Time: 00:00 | Remaining Time: ?

Launching job: bars_near_london_bridge_3_offline_fg_materialization
Job started successfully, you can follow the progress at 
https://c.app.hopsworks.ai/p/25749/jobs/named/bars_near_london_bridge_3_offline_fg_materialization/executions
Historical data backfilled successfully, with added label for busyness.


In [15]:
update_realtime_data()

Performing venue search
Venue search initiated. Progress URL: https://besttime.app/api/v1/venues/progress?job_id=07d93a4e-39d0-4bf9-836a-c2c380a09f67&collection_id=col_e28024440353404c9f554a79dec089c6&format=all
Retrieving venue search results...
Found 34 venues.
Extracting historical data...


Uploading Dataframe: 0.00% |          | Rows 0/3696 | Elapsed Time: 00:00 | Remaining Time: ?

Launching job: bars_near_london_bridge_3_offline_fg_materialization
Job started successfully, you can follow the progress at 
https://c.app.hopsworks.ai/p/25749/jobs/named/bars_near_london_bridge_3_offline_fg_materialization/executions
Real-time data updated successfully.


In [19]:
def main():
    # Call backfill_historical_data() once to backfill
    # Uncomment the next line to perform backfill
    backfill_historical_data()

    # Call update_realtime_data() periodically to update with real-time data
    update_realtime_data()

if __name__ == "__main__":
    main()


Starting backfill
Performing venue search
Venue search initiated. Progress URL: https://besttime.app/api/v1/venues/progress?job_id=a384dda0-7844-4008-9169-378748fb1a7d&collection_id=col_8aa381790a1b4cacb2a889220690c474&format=all
Retrieving venue search results...
Found 37 venues.
Extracting historical data...
Feature Group created successfully, explore it at 
https://c.app.hopsworks.ai:443/p/25749/fs/25669/fg/886123
2024-06-07 11:16:37,335 WARNING: DeprecationWarning: The `ipykernel.comm.Comm` class has been deprecated. Please use the `comm` module instead.For creating comms, use the function `from comm import create_comm`.



Uploading Dataframe: 0.00% |          | Rows 0/4032 | Elapsed Time: 00:00 | Remaining Time: ?

Launching job: bars_near_london_bridge_3_offline_fg_materialization
Job started successfully, you can follow the progress at 
https://c.app.hopsworks.ai/p/25749/jobs/named/bars_near_london_bridge_3_offline_fg_materialization/executions
Historical data backfilled successfully, with added label for busyness.
